## MLflow's Model Registry

In [4]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# client.list_experiments()
exps = client.search_experiments()  # optionally: view_type=ViewType.ALL
for e in exps:
    print(e.experiment_id, e.name)

3 my-cool-experiment
2 nyc-taxi-experiment


In [ ]:
client.create_experiment(name="my-cool-experiment")

Let's check the latest versions for the experiment with id `2`...

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 97ea03dddba74403975e5d6e0d1e222d, rmse: 6.3087
run id: b6ac67af0dc54a1ab87fc6e6c7bd65be, rmse: 6.3087
run id: 2157e80584aa4d49accd73e9119b8105, rmse: 6.3087
run id: 580e669843954c88a52e9e599e570380, rmse: 6.3137
run id: 6539e60a8b6345699062c0c70b6e38ec, rmse: 6.3210


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [10]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [13]:
run_id = "b6ac67af0dc54a1ab87fc6e6c7bd65be"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2026/02/24 20:04:26 WARNING mlflow.tracking._model_registry.fluent: Run with id b6ac67af0dc54a1ab87fc6e6c7bd65be has no artifacts at artifact path 'models_mlflow', registering model based on models:/m-3a50a0508a2b43068ff772c24ff7eb88 instead


Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1771963466153, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1771963466153, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='b6ac67af0dc54a1ab87fc6e6c7bd65be', run_link=None, source='models:/m-3a50a0508a2b43068ff772c24ff7eb88', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [14]:
model_uri

'runs:/b6ac67af0dc54a1ab87fc6e6c7bd65be/models_mlflow'

In [15]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


/tmp/ipykernel_55187/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [22]:
model_name = "nyc-taxi-regressor"
model_version = 3
new_stage = "Staging"

# The new, modern way to assign a lifecycle stage using Aliases
client.transition_model_version_stage(
    name=model_name, 
    stage=new_stage,
    version=model_version,
    archive_existing_versions=False
)

/tmp/ipykernel_55187/2966664206.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=['staging'], creation_timestamp=1771963466153, current_stage='Staging', deployment_job_state=None, description='The model version 3 was transitioned to Staging on 2026-02-24', last_updated_timestamp=1771964254158, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='b6ac67af0dc54a1ab87fc6e6c7bd65be', run_link=None, source='models:/m-3a50a0508a2b43068ff772c24ff7eb88', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [23]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=['staging'], creation_timestamp=1771963466153, current_stage='Staging', deployment_job_state=None, description='The model version 3 was transitioned to Staging on 2026-02-24', last_updated_timestamp=1771964257935, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='b6ac67af0dc54a1ab87fc6e6c7bd65be', run_link=None, source='models:/m-3a50a0508a2b43068ff772c24ff7eb88', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [38]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [30]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [32]:
run_id = "97ea03dddba74403975e5d6e0d1e222d"
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/joeyge1218/MLOps-zoomcamp/02-experiment-tracking/preprocessor'

In [33]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [34]:
X_test = preprocess(df, dv)

In [35]:
target = "duration"
y_test = df[target].values

In [39]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 289 ms, sys: 4.57 ms, total: 294 ms
Wall time: 292 ms


{'rmse': 6.608690086957033}

In [40]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 31.9 s, sys: 117 ms, total: 32.1 s
Wall time: 8.31 s


{'rmse': 6.262885819169815}

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_55187/3610659809.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


MlflowException: Model Version (name=nyc-taxi-regressor, version=4) not found